# Word2Vec con Gensim

## Instalación de librerías y carga de dataset

In [1]:
!pip install datasets

In [2]:
from gensim.models import Word2Vec
import pandas as pd
import re
from gensim.parsing.preprocessing import strip_punctuation, strip_numeric, strip_short, stem_text
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

-------------------------

In [ ]:
from datasets import load_dataset

dataset_corpus = load_dataset("large_spanish_corpus", "ParaCrawl")

In [ ]:
# elegimos de train el primer millón de ejemplos
subset = dataset_corpus['train'].select(range(1000000))
subset[:3]

In [ ]:
import nltk
from nltk.tokenize import word_tokenize

## Procesamiento de texto

In [ ]:
def clean_text(batch_sentences):
    # extrae el texto de la entrada
    text_list = batch_sentences['text']
    cleaned_text_list = []
    for text in text_list:
        # Convierte el texto a minúsculas
        text = text.lower()

        # Elimina URLs
        text = re.sub(r'https?://\S+|www\.\S+', '', text, flags=re.MULTILINE)

        # Elimina las menciones @ y '#' de las redes sociales
        text = re.sub(r'@\w+|#\w+', '', text)

        # Elimina los caracteres de puntuación
        text = strip_punctuation(text)

        # Elimina los números
        text = strip_numeric(text)

        # Elimina las palabras cortas
        text = strip_short(text, minsize=2)

        # Elimina las palabras comunes (stop words)
        stop_words = set(stopwords.words('spanish'))
        
        # Tokeniza el texto por cada oración
        tokens = word_tokenize(text)
        
        # elimina los stopwords (palabras comunes)
        
        filtered_text_st =[word for word in tokens if word not in stop_words]
        
        cleaned_text_list.append(filtered_text_st)



    # Devuelve el texto limpio
    return {
        'text' : cleaned_text_list
    }



In [ ]:
sentence_corpus = subset.map(clean_text, batched=True)

Map: 100%|██████████| 1000000/1000000 [01:57<00:00, 8481.78 examples/s]


In [ ]:
sentence_corpus['text'][:3]

[['lavado',
  'cerebro',
  'través',
  'medios',
  'comunicación',
  'amenaza',
  'fuerza',
  'través',
  'militares'],
 ['constante',
  'aluvión',
  'doble',
  'cañón',
  'requiriendo',
  'complicidad',
  'seres',
  'humanos',
  'reprimir',
  'engañar',
  'semejantes',
  'tan',
  'cacareada',
  'magia',
  'rápidamente',
  'desvanecería',
  'disiparía'],
 ['realidad',
  'nuevo',
  'om',
  'sólo',
  'puede',
  'mantener',
  'ilusión',
  'supremacía',
  'mágica',
  'siempre',
  'reprima',
  'desvíe',
  'potencial',
  'humano',
  'mora',
  'verdadera',
  'magia',
  'decir',
  'capacidad',
  'innata',
  'especie',
  'magia',
  'interactiva',
  'poderes',
  'animación',
  'diosa',
  'planetaria']]

## Carga y uso de modelo de embeddings Word2Vec

In [ ]:
model = Word2Vec(sentence_corpus['text'], vector_size=100, window=5, min_count=2, workers=4, sg=1)

# Podemos guardar el modelo para uso futuro
model.save("word2vec.model")

In [ ]:
# probamos la representación vectorial de una palabra
model.wv['rey']

array([-0.01775959,  0.06014758,  1.1201856 ,  0.85155106,  0.04822263,
       -0.21653563,  0.29199094,  0.64293474, -0.18336612,  0.09501904,
       -0.8040728 , -0.52166975,  0.3661424 ,  0.24856174, -0.6375646 ,
       -0.22724645,  0.33896494, -0.918988  ,  0.48282984,  0.5330522 ,
       -0.14381169,  0.8380222 ,  0.42422083,  0.06555478,  0.70205855,
       -0.19463238, -0.05770623,  0.07866146, -0.3106073 ,  0.008195  ,
       -0.5552841 ,  0.46386743,  0.07576317, -0.5845578 , -0.57834846,
        0.4358849 , -0.36418912, -0.41799632, -0.2569206 , -0.6127153 ,
       -0.2619601 , -0.05185571, -0.34142482,  0.09443165, -0.07691523,
        0.21536507, -0.8239192 , -0.04915377, -0.03385297,  0.1636164 ,
        0.6041658 , -0.6106307 ,  0.01406347,  0.09174182, -0.47830933,
       -0.21365976, -0.05881346, -0.46212432,  0.04515149,  0.4269978 ,
       -0.0901477 ,  0.0605472 ,  0.13558947, -0.49378043,  0.06787198,
        0.8284292 , -0.33021188,  0.46007028, -0.570047  ,  0.01

In [ ]:
# con vw.most_similar podemos encontrar las palabras más similares a una dada
model.wv.most_similar('comida',topn=3)

[('comer', 0.7584296464920044),
 ('comidas', 0.7391618490219116),
 ('sabrosa', 0.7338200211524963)]

## similitud de palabras y preparacion para visualizar

In [ ]:
model.wv.most_similar('reina',topn=3) 

[('sofía', 0.7716379165649414),
 ('peinador', 0.735051155090332),
 ('semíramis', 0.7190153002738953)]

In [ ]:
# separar los vectores y las palabras
word_and_vectors = model.wv
vectors = model.wv.vectors
words = model.wv.index_to_key

In [ ]:
len(words)


128768

## Almacenamiento de embeddings

In [ ]:
df_tsv = pd.DataFrame(vectors)
df_tsv.to_csv('vectors.tsv', sep='\t', index=False, header=False)

In [ ]:
df_tsv = pd.DataFrame(words)
df_tsv.to_csv('words.tsv', sep='\t', index=False, header=False)